In [110]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, Lars, LassoLars
from sklearn.linear_model import HuberRegressor, OrthogonalMatchingPursuit
from sklearn.dummy import DummyRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb

from xgboost import XGBRegressor



C:\Users\ahayd\AppData\Roaming\Python\Python312\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [126]:
# Veri hazırlığı
data = pd.read_csv('datathon-2024/tag-data.csv')

X = data.drop(columns=['id', 'puan']).fillna(21)
y = data['puan'].fillna(21)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


In [123]:
from sklearn.preprocessing import StandardScaler

# Standardizasyon uygulama
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)  # Eğitim verisini fit ve transform et
X_test_scaled = scaler.transform(X_test)        # Test verisini sadece transform et


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   puan             65125 non-null  int64  
 1   id               65125 non-null  int64  
 2   Sehir Tag        65125 non-null  int64  
 3   Universite Tag   65125 non-null  int64  
 4   Bolum            65125 non-null  int64  
 5   ikinci okul      65125 non-null  int64  
 6   lise bolum       65125 non-null  int64  
 7   Universite Turu  65125 non-null  int64  
 8   Sinif            65125 non-null  int64  
 9   Not Ortalamasi   65125 non-null  int64  
 10  lise tur         65125 non-null  int64  
 11  anne egitim      65125 non-null  int64  
 12  baba egitim      65125 non-null  int64  
 13  kulup            65125 non-null  int64  
 14  stk proje        65125 non-null  int64  
 15  girisim          65125 non-null  int64  
 16  ingilizce        65125 non-null  int64  
 17  cinsiyet    

In [124]:
model = XGBRegressor(
    alpha=0.2,  
    reg_lambda=0.2, 
    subsample=0.7, 
    colsample_bytree=0.3,  
    learning_rate=0.05, 
    objective='reg:squarederror',  
    max_depth=16,  
    min_child_weight=30, 
    gamma=1e-6, 
    n_estimators=2000,  
    eval_metric='rmse',  
    early_stopping_rounds=15
)

XGB = model.fit(
    X_train_scaled, 
    y_train, 
    eval_set=[(X_test_scaled, y_test)], 
    verbose=True  
)


[0]	validation_0-rmse:18.02166
[1]	validation_0-rmse:17.52921
[2]	validation_0-rmse:17.07684
[3]	validation_0-rmse:16.97450
[4]	validation_0-rmse:16.43492
[5]	validation_0-rmse:16.12034
[6]	validation_0-rmse:15.76561
[7]	validation_0-rmse:15.45432
[8]	validation_0-rmse:15.36272
[9]	validation_0-rmse:15.25896
[10]	validation_0-rmse:15.12590
[11]	validation_0-rmse:14.90268
[12]	validation_0-rmse:14.66800
[13]	validation_0-rmse:14.48939
[14]	validation_0-rmse:14.25423
[15]	validation_0-rmse:14.15718
[16]	validation_0-rmse:14.09468
[17]	validation_0-rmse:14.02203
[18]	validation_0-rmse:13.96899
[19]	validation_0-rmse:13.73676
[20]	validation_0-rmse:13.42399
[21]	validation_0-rmse:13.37087
[22]	validation_0-rmse:13.11754
[23]	validation_0-rmse:13.07541
[24]	validation_0-rmse:12.91943
[25]	validation_0-rmse:12.65278
[26]	validation_0-rmse:12.50845
[27]	validation_0-rmse:12.29962
[28]	validation_0-rmse:12.26217
[29]	validation_0-rmse:12.22914
[30]	validation_0-rmse:12.08131
[31]	validation_0-

In [153]:
y_pred = model.predict(X_test)


In [154]:
# Tahminleri 0-100 aralığına sınırlama
y_pred = np.clip(y_pred, 0, 100)

# Performans değerlendirmesi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
# r2 = model.score(X_test, y_test)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"RMSE: {rmse}")
# print(f"R² Score: {r2}")

Mean Absolute Error: 6.30666396295712
Mean Squared Error: 69.62826179594437
RMSE: 8.344355085681839


In [106]:
df = pd.read_csv('datathon-2024/tag-test-data.csv')  # Verinizi yükleyin
df = df.drop(columns='id')

In [107]:
pred = model.predict(df)


In [156]:
pred = pd.DataFrame(pred)

pred.to_csv("sample_submission.csv", index=True)

In [155]:
model = CatBoostRegressor(learning_rate=0.15, depth=6, iterations=500, verbose=1)

model.fit(X_train, y_train)

0:	learn: 16.2651478	total: 11.1ms	remaining: 5.55s
1:	learn: 14.7775442	total: 19.3ms	remaining: 4.82s
2:	learn: 13.6558285	total: 69.7ms	remaining: 11.5s
3:	learn: 12.7748634	total: 89.3ms	remaining: 11.1s
4:	learn: 12.0894941	total: 97.2ms	remaining: 9.63s
5:	learn: 11.5758461	total: 114ms	remaining: 9.4s
6:	learn: 11.0680967	total: 128ms	remaining: 9.01s
7:	learn: 10.6797286	total: 134ms	remaining: 8.23s
8:	learn: 10.4100970	total: 139ms	remaining: 7.61s
9:	learn: 10.1954724	total: 145ms	remaining: 7.12s
10:	learn: 9.9826440	total: 151ms	remaining: 6.71s
11:	learn: 9.8462312	total: 156ms	remaining: 6.35s
12:	learn: 9.6977283	total: 161ms	remaining: 6.02s
13:	learn: 9.5822930	total: 166ms	remaining: 5.75s
14:	learn: 9.4876785	total: 171ms	remaining: 5.51s
15:	learn: 9.4053230	total: 176ms	remaining: 5.32s
16:	learn: 9.3424253	total: 181ms	remaining: 5.15s
17:	learn: 9.2546185	total: 187ms	remaining: 5s
18:	learn: 9.2153376	total: 193ms	remaining: 4.88s
19:	learn: 9.1452792	total: 19

In [125]:

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'Least Angle Regression': Lars(),
    'Lasso Least Angle Regression': LassoLars(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Extra Trees Regressor': ExtraTreesRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K Neighbors Regressor': KNeighborsRegressor(),
    'Huber Regressor': HuberRegressor(),
    'Orthogonal Matching Pursuit': OrthogonalMatchingPursuit(),
    'Dummy Regressor': DummyRegressor(strategy='mean'),
    'Extreme Gradient Boosting': xgb.XGBRegressor(),
    'Light Gradient Boosting Machine': lgb.LGBMRegressor(),
    'CatBoost Regressor': CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, verbose=0),
    'Hist Gradient Boosting Regressor': HistGradientBoostingRegressor()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = rmse
    print(f'{name}: RMSE = {rmse}')

results_df = pd.DataFrame(list(results.items()), columns=['Model', 'RMSE'])
print(results_df.sort_values(by='RMSE'))

Linear Regression: RMSE = 12.440109771875516
Ridge Regression: RMSE = 12.440111653610082
Lasso Regression: RMSE = 13.115539872733548
Elastic Net: RMSE = 14.861648917849674
Bayesian Ridge: RMSE = 12.440123057471057
Least Angle Regression: RMSE = 12.440109771875516
Lasso Least Angle Regression: RMSE = 13.11554696434786
Gradient Boosting Regressor: RMSE = 8.933812164141195
Random Forest Regressor: RMSE = 8.779624493864773
Extra Trees Regressor: RMSE = 9.111127583283976
AdaBoost Regressor: RMSE = 11.895463595191803
Decision Tree Regressor: RMSE = 12.420057604280391
K Neighbors Regressor: RMSE = 16.95958211542186


C:\Users\ahayd\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Huber Regressor: RMSE = 13.672625293480824
Orthogonal Matching Pursuit: RMSE = 18.182695134691404
Dummy Regressor: RMSE = 18.236070548643276
Extreme Gradient Boosting: RMSE = 8.466590903820673
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002506 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 708
[LightGBM] [Info] Number of data points in the train set: 55356, number of used features: 21
[LightGBM] [Info] Start training from score 32.075547
Light Gradient Boosting Machine: RMSE = 8.423248513751993
CatBoost Regressor: RMSE = 8.333675774218685
Hist Gradient Boosting Regressor: RMSE = 8.424122154664715
                               Model       RMSE
18                CatBoost Regressor   8.333676
17   Light Gradient Boosting Machine   8.423249
19  Hist Gradie